In [3]:
from forcha.models.pytorch.mnist import MNIST_CNN
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

import os

In [10]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Data
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307), (0.3081)),
])

testset = torchvision.datasets.MNIST(
    root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=32, shuffle=False, num_workers=2)

In [11]:
# Model
print('==> Building model..')

net = MNIST_CNN()
net.to(device)

criterion = nn.CrossEntropyLoss()

==> Building model..


In [12]:
# Test function
def test():
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
        
        return (test_loss, batch_idx, correct, total)

In [13]:
# Testing model of each round
path = os.path.join(f'/home/mzuziak/snap/snapd-desktop-integration/83/Documents/Forcha/forcha/tests/test_cases/simple_fedopt/archiver_from_11_10_2023__11_39_29/models/orchestrator')
list_of_models = os.listdir(path)

for model in list_of_models:
    weights = os.path.join(path, model)
    print(f"===> Loaded model {model}")
    net.load_state_dict(torch.load(weights))
    test_loss, batch_idx, correct, total = test()
    # Print test loss and accuracy
    print(f"===>RESULTS FOR MODEL {model}<===")
    print(f'Test Loss: {test_loss / (batch_idx +1):.3f}')
    print(f'Test Accuracy: {100. * correct / total:.2f}%')

Test Loss: 0.150
Test Accuracy: 98.71%
